In [1]:
import cv2
import os
import re
import numpy as np

In [2]:
def find_largest_contour(contours):
    """入力contourのlistから最大長のcontourを返す"""
    max_idx = -1
    max_area = 0
    for i, cnt in enumerate(contours):
        if cv2.contourArea(cnt) > max_area:
            max_idx = i
            max_area = cv2.contourArea(cnt)
    cnt = contours[max_idx]
    return cnt

def remove_noise(img, largest_cnt):
    """最大長のcontourでマスクし，ノイズを除去したimgを返す"""
    blk = np.zeros(img.shape, np.uint8)
    mask = cv2.drawContours(blk, [largest_cnt], 0, (255, 255, 255), -1)
    noise_removed_img = cv2.bitwise_and(img, mask)
    return noise_removed_img

### 入出力先の指定
* 入力: segmentation画像
* 出力: ノイズ除去されたsegmentation画像

In [3]:
SEG = './seg'
DST = './noise_removed'
fnames = sorted(os.listdir(SEG))
fnames[:5]

['138-P-1_211122.png',
 '138-P-1_211124.png',
 '138-P-1_211126.png',
 '138-P-1_211127.png',
 '138-P-1_211129.png']

### ノイズの除去の実行

In [4]:
for fname in fnames:
    """contoursを取得"""
    img = cv2.imread(os.path.join(SEG, fname))
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    ret,thresh = cv2.threshold(gray,1,255,0)
    contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)
    """contoursを元にノイズを除去"""
    largest_cnt = find_largest_contour(contours)
    noise_removed_img = remove_noise(img, largest_cnt)
    cv2.imwrite(os.path.join(DST, fname), noise_removed_img)